In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from datasets import load_dataset

dataset = load_dataset('imdb')


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:

# Checking data
dataset["unsupervised"][0]


{'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
 'label': -1}

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

<h1>With Logistic regression </h1>

In [5]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

X_train = vectorizer.fit_transform(dataset['train']['text'])
y_train = dataset['train']['label']

X_test = vectorizer.transform(dataset['test']['text'])
y_test = dataset['test']['label']


In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
classifier = LogisticRegression(max_iter=1000)

In [8]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [9]:
from sklearn.metrics import accuracy_score, classification_report

predictions = classifier.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, target_names=["Negative", "Positive"]))


Accuracy: 0.87944
              precision    recall  f1-score   support

    Negative       0.88      0.88      0.88     12500
    Positive       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



<h1>With RNN network</h1>

In [10]:
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

In [11]:
X_train_texts = dataset['train']['text']
y_train2 = np.array(dataset['train']['label'])

X_test_texts = dataset['test']['text']
y_test2 = np.array(dataset['test']['label'])


In [12]:
vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")


In [13]:
tokenizer.fit_on_texts(X_train_texts)

In [14]:
X_train_seq = tokenizer.texts_to_sequences(X_train_texts)
X_test_seq = tokenizer.texts_to_sequences(X_test_texts)


In [15]:
max_length = 200
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
import numpy as np

X_train_padded = np.array(X_train_padded, dtype=np.int32)
y_train2 = np.array(y_train2, dtype=np.int32)
X_test_padded = np.array(X_test_padded, dtype=np.int32)
y_test2 = np.array(y_test2, dtype=np.int32)


In [19]:

# Train the model
num_epochs = 5
history = model.fit(X_train_padded, y_train2,
                    epochs=num_epochs,
                    validation_data=(X_test_padded, y_test2),
                    batch_size=128)


Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 69s 337ms/step - accuracy: 0.5134 - loss: 0.6910 - val_accuracy: 0.5000 - val_loss: 0.6974
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 65s 332ms/step - accuracy: 0.5167 - loss: 0.6905 - val_accuracy: 0.5249 - val_loss: 0.6932
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 65s 334ms/step - accuracy: 0.5237 - loss: 0.6885 - val_accuracy: 0.5690 - val_loss: 0.6711
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 66s 335ms/step - accuracy: 0.5777 - loss: 0.6583 - val_accuracy: 0.4936 - val_loss: 0.6936
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 65s 334ms/step - accuracy: 0.5121 - loss: 0.6925 - val_accuracy: 0.6725 - val_loss: 0.6263


In [20]:
import numpy as np

def predict_logistic(text, vectorizer, model):
    """
    Converts raw text into TF-IDF features and predicts sentiment using a Logistic Regression model.

    Parameters:
    - text (str): The input text.
    - vectorizer (TfidfVectorizer): Pre-trained TF-IDF vectorizer.
    - model (LogisticRegression): Trained Logistic Regression model.

    Returns:
    - str: "Positive" or "Negative"
    """
    # Convert text to TF-IDF vector
    text_tfidf = vectorizer.transform([text])
    
    # Predict sentiment (0 or 1)
    prediction = model.predict(text_tfidf)[0]
    
    return "Positive" if prediction == 1 else "Negative"

# Example usage (after training the model):
# sentiment = predict_logistic("The movie was amazing!", tfidf_vectorizer, logistic_model)
# print(sentiment)


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_rnn(text, tokenizer, model, max_length):
    """
    Converts raw text into a padded sequence for an RNN model and predicts sentiment.

    Parameters:
    - text (str): The input text.
    - tokenizer (Tokenizer): Pre-trained Keras tokenizer.
    - model (tf.keras.Model): Trained RNN/LSTM model.
    - max_length (int): The maximum sequence length for padding.

    Returns:
    - str: "Positive" or "Negative"
    """
    # Convert text to a sequence of word indices
    text_seq = tokenizer.texts_to_sequences([text])
    
    # Pad sequence to match the required input length
    text_padded = pad_sequences(text_seq, maxlen=max_length, padding='post')

    # Predict sentiment probability
    prediction = model.predict(text_padded)[0][0]
    
    return "Positive" if prediction >= 0.5 else "Negative"

# Example usage (after training the model):
# sentiment = predict_rnn("The movie was terrible!", tokenizer, rnn_model, max_length)
# print(sentiment)


In [22]:
text_input = "This was amazing"

# Logistic Regression Prediction
logistic_result = predict_logistic(text_input, vectorizer, classifier)
print("Logistic Regression Prediction:", logistic_result)

# RNN (LSTM) Prediction
rnn_result = predict_rnn(text_input, tokenizer, model, max_length)
print("RNN Prediction:", rnn_result)


Logistic Regression Prediction: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
RNN Prediction: Negative
